### __未经测试，不确认正确性__
## VGG16 with 17flowers

In [56]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torchvision import datasets, transforms
from torch.optim.lr_scheduler import StepLR
import torchvision
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import seaborn as sns

In [37]:
# Download Oxford 17 Category Flower Dataset
!wget http://www.robots.ox.ac.uk/~vgg/data/flowers/17/17flowers.tgz -O 17flowers.tgz
!tar -xvzf 17flowers.tgz

--2022-10-16 20:39:13--  http://www.robots.ox.ac.uk/~vgg/data/flowers/17/17flowers.tgz
正在解析主机 www.robots.ox.ac.uk (www.robots.ox.ac.uk)... 129.67.94.2
正在连接 www.robots.ox.ac.uk (www.robots.ox.ac.uk)|129.67.94.2|:80... 已连接。
已发出 HTTP 请求，正在等待回应... 301 Moved Permanently
位置：https://www.robots.ox.ac.uk/~vgg/data/flowers/17/17flowers.tgz [跟随至新的 URL]
--2022-10-16 20:39:14--  https://www.robots.ox.ac.uk/~vgg/data/flowers/17/17flowers.tgz
正在连接 www.robots.ox.ac.uk (www.robots.ox.ac.uk)|129.67.94.2|:443... 已连接。
已发出 HTTP 请求，正在等待回应... 200 OK
长度：60270631 (57M) [application/x-gzip]
正在保存至: “17flowers.tgz”

17flowers.tgz       100%[===================>]  57.48M  12.0MB/s  用时 6.0s      

2022-10-16 20:39:21 (9.59 MB/s) - 已保存 “17flowers.tgz” [60270631/60270631])

x jpg/files.txt
x jpg/files.txt~
x jpg/image_0001.jpg
x jpg/image_0002.jpg
x jpg/image_0003.jpg
x jpg/image_0004.jpg
x jpg/image_0005.jpg
x jpg/image_0006.jpg
x jpg/image_0007.jpg
x jpg/image_0008.jpg
x jpg/image_0009.jpg
x jpg/image_0010.jpg
x jp

In [58]:
# Build VGG-16 model
class VGG(nn.Module):
    def __init__(self):
        super(VGG, self).__init__()
        self.conv1 = nn.Conv2d(3, 64, kernel_size=3, padding=1)
        self.conv2 = nn.Conv2d(64, 64, kernel_size=3, padding=1)
        self.conv3 = nn.Conv2d(64, 128, kernel_size=3, padding=1)
        self.conv4 = nn.Conv2d(128, 128, kernel_size=3, padding=1)
        self.conv5 = nn.Conv2d(128, 256, kernel_size=3, padding=1)
        self.conv6 = nn.Conv2d(256, 256, kernel_size=3, padding=1)
        self.conv7 = nn.Conv2d(256, 256, kernel_size=3, padding=1)
        self.conv8 = nn.Conv2d(256, 512, kernel_size=3, padding=1)
        self.conv9 = nn.Conv2d(512, 512, kernel_size=3, padding=1)
        self.conv10 = nn.Conv2d(512, 512, kernel_size=3, padding=1)
        self.conv11 = nn.Conv2d(512, 512, kernel_size=3, padding=1)
        self.conv12 = nn.Conv2d(512, 512, kernel_size=3, padding=1)
        self.conv13 = nn.Conv2d(512, 512, kernel_size=3, padding=1)
        self.fc1 = nn.Linear(512 * 7 * 7, 4096)
        self.fc2 = nn.Linear(4096, 4096)
        self.fc3 = nn.Linear(4096, 17)

    def forward(self, x):
        x = F.relu(self.conv1(x))
        x = F.relu(self.conv2(x))
        x = F.max_pool2d(x, 2)
        x = F.relu(self.conv3(x))
        x = F.relu(self.conv4(x))
        x = F.max_pool2d(x, 2)
        x = F.relu(self.conv5(x))
        x = F.relu(self.conv6(x))
        x = F.relu(self.conv7(x))
        x = F.max_pool2d(x, 2)
        x = F.relu(self.conv8(x))
        x = F.relu(self.conv9(x))
        x = F.relu(self.conv10(x))
        x = F.max_pool2d(x, 2)
        x = F.relu(self.conv11(x))
        x = F.relu(self.conv12(x))
        x = F.relu(self.conv13(x))
        x = F.max_pool2d(x, 2)
        x = x.view(-1, 512 * 7 * 7)
        x = F.relu(self.fc1(x))
        x = F.dropout(x, training=self.training)
        x = F.relu(self.fc2(x))
        x = F.dropout(x, training=self.training)
        x = self.fc3(x)
        return F.log_softmax(x, dim=1)

In [31]:
# Train the model
def train(model, device, train_loader, optimizer, epoch):
    model.train()
    for batch_idx, (data, target) in enumerate(train_loader):
        data, target = data.to(device), target.to(device)
        optimizer.zero_grad()
        output = model(data)
        loss = F.nll_loss(output, target)
        loss.backward()
        optimizer.step()
        if batch_idx % 10 == 0:
            print('Train Epoch: {} [{}/{} ({:.0f}%)]\tLoss: {:.6f}'.format(
                epoch, batch_idx * len(data), len(train_loader.dataset),
                100. * batch_idx / len(train_loader), loss.item()))

In [32]:
# Test the model
def test(model, device, test_loader):
    model.eval()
    test_loss = 0
    correct = 0
    with torch.no_grad():
        for data, target in test_loader:
            data, target = data.to(device), target.to(device)
            output = model(data)
            test_loss += F.nll_loss(output, target, reduction='sum').item()  # sum up batch loss
            pred = output.argmax(dim=1, keepdim=True)  # get the index of the max log-probability
            correct += pred.eq(target.view_as(pred)).sum().item()

    test_loss /= len(test_loader.dataset)

    print('Test set: Average loss: {:.4f}, Accuracy: {}/{} ({:.0f}%)'.format(
        test_loss, correct, len(test_loader.dataset),
        100. * correct / len(test_loader.dataset)))

In [38]:
# transform the images to tensors
transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize((0.485, 0.456, 0.406), (0.229, 0.224, 0.225))
])

In [54]:
# Split the images in ./jpg into 17 folders
!cd ./jpg && for i in {1..17}; do mkdir $i; done
# Move image_xxxx.jpg to folder (((xxxx-1)//80) + 1)
!cd ./jpg && for i in {0001..1360}; do mv "image_$i.jpg" $(((i-1)/80 + 1)); done


In [57]:
# prepare the Oxford 17 Category Flower Dataset which is in the folder jpg
# the dataset is divided into 80% training and 20% testing
Dataset = torchvision.datasets.ImageFolder(root='./jpg', transform=transform)
train_size = int(0.8 * len(Dataset))
test_size = len(Dataset) - train_size
train_dataset, test_dataset = torch.utils.data.random_split(Dataset, [train_size, test_size])

In [ ]:
# Prepare the train_dataset
train_loader = torch.utils.data.DataLoader(train_dataset, batch_size=64, shuffle=True)
# Prepare the test_dataset
test_loader = torch.utils.data.DataLoader(test_dataset, batch_size=64, shuffle=True)

In [61]:
# Run the VGG model with 17 classes
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = VGG().to(device)

epochs = 20
lr = 0.001

# Training
optimizer = optim.SGD(model.parameters(), lr=lr, momentum=0.9)
for epoch in range(epochs):
    print("Epoch ", epoch, ": ")
    train(model, device, train_loader, optimizer, epoch)
    test(model, device, test_loader)


Epoch  1 : 


KeyboardInterrupt: 